## [ONNX Runtime](https://github.com/microsoft/onnxruntime/)

In [1]:
import numpy as np
import onnxruntime as ort

In [2]:
ort_session = ort.InferenceSession('resnet18.onnx')

In [3]:
model_input = {'model_input1': np.random.randn(1, 3, 256, 256).astype(np.float32)}

In [4]:
%%timeit -r 100
ort_session.run(['output1'], model_input)

47.9 ms ± 4.04 ms per loop (mean ± std. dev. of 100 runs, 10 loops each)


In [5]:
ort_out = ort_session.run(['output1'], model_input)[0][0]
print(ort_out.shape)

(1000,)


## [Torch Script](https://pytorch.org/docs/stable/jit.html)

In [6]:
import torch
from torchvision.models import resnet18

In [7]:
inp = torch.from_numpy(model_input['model_input1'])
inp.size()

torch.Size([1, 3, 256, 256])

In [8]:
model = resnet18(pretrained=True)
model = model.eval()
model = torch.jit.trace(model, inp)

In [9]:
%%timeit -r 100

with torch.no_grad():
    model(inp)

19.1 ms ± 28.7 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [10]:
with torch.no_grad():
    torch_out = model(inp).detach().numpy()[0]
print(torch_out.shape)

(1000,)


In [11]:
for conf_lvl in (1e-3, 1e-3 / 2, 1e-4, 1e-4 / 2, 1e-5, 1e-5 / 2, 1e-6, 1e-6 / 2, 1e-7):
    print(conf_lvl, np.allclose(ort_out, torch_out, atol=conf_lvl))

0.001 True
0.0005 True
0.0001 True
5e-05 True
1e-05 True
5e-06 True
1e-06 False
5e-07 False
1e-07 False
